In [1]:

#SUBMIT_ARGS = "--packages com.databricks:spark-csv_2.11-1.5.0 pyspark-shell"
#os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [1]:
#import os
#os.environ["PYSPARK_PYTHON"]='/Users/diogomesquita/anaconda/envs/py3.6/bin/python'
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.sql.functions import *
import utils

In [11]:
# LOAD DATA
spark = SparkSession \
        .builder \
        .appName("Nulls and Outliers Detection") \
        .getOrCreate()

collection = ['GROUP7/bss9-579f.tsv', 'GROUP7/5gde-fmj3.tsv', 'GROUP7/66be-66yr.tsv', 'GROUP7/7crd-d9xh.tsv', 'GROUP7/dvzp-h4k9.tsv']
df = spark.read.csv(path = collection[0], header = True,inferSchema = True, sep='\t')

In [12]:
df_test = spark.createDataFrame([('923,122,344.12','187,988', 'Porto', 10),('555,666,','888,444', "Coimbra",20), ('555,666.10,','888$,444', "Coimbra",11), ('555,666,','888,444', "Coimbra",8)], ['s1','s2', 's3', 'myint'])

In [13]:
replaced_df = df
for col, dtype in replaced_df.dtypes:
    print(f"### {col} ###")
    if 'string' in dtype:
        print(f"\tIs string!")
        tmp_col = f"{col}_temp"
        print(f"\t{replaced_df.select(col).limit(5).rdd.map(lambda x: x[0]).collect()}")
        replaced_df = replaced_df.withColumn(tmp_col, utils.replace_commas_symbols_udf(replaced_df[col]).cast(DoubleType()))
        if utils.is_double(replaced_df, tmp_col):
            print(f"\tIs double actually!")
            replaced_df = replaced_df.withColumn(col, replaced_df[tmp_col])
        replaced_df = replaced_df.drop(tmp_col)
        print(f"\t{replaced_df.select(col).limit(5).rdd.map(lambda x: x[0]).collect()}")

### comparable_rental_2_address ###
	Is string!
	['99 GOLD STREET', '18 SIDNEY PLACE', '99 GOLD STREET', '337 STATE STREET', '337 STATE STREET']
	['99 GOLD STREET', '18 SIDNEY PLACE', '99 GOLD STREET', '337 STATE STREET', '337 STATE STREET']
### brooklyn_condominiums_comparable_properties_address ###
	Is string!
	['1 MAIN STREET', '31 WASHINGTON STREET', '133 WATER STREET', '50 BRIDGE STREET', '4 WATER STREET']
	['1 MAIN STREET', '31 WASHINGTON STREET', '133 WATER STREET', '50 BRIDGE STREET', '4 WATER STREET']
### comparable_rental_2_gross_sqft ###
### comparable_rental_3_estimated_gross_income ###
	Is string!
	['744,258', '875,152', None, '525,261', '1,669,215']
	['744,258', '875,152', None, '525,261', '1,669,215']
### brooklyn_condominiums_comparable_properties_estimated_expense ###
### comparable_rental_2_building_classification ###
	Is string!
	['D5-ELEVATOR', 'D9-ELEVATOR', 'D5-ELEVATOR', 'D1-ELEVATOR', 'D1-ELEVATOR']
	['D5-ELEVATOR', 'D9-ELEVATOR', 'D5-ELEVATOR', 'D1-ELEVATOR', '

In [292]:
replaced_df.dtypes

[('s1', 'double'), ('s2', 'double'), ('s3', 'string'), ('myint', 'bigint')]

In [293]:
replaced_df.collect()

[Row(s1=923122344.0, s2=187988.0, s3='Porto', myint=10),
 Row(s1=555666.0, s2=888444.0, s3='Coimbra', myint=20),
 Row(s1=555666.0, s2=None, s3='Coimbra', myint=11),
 Row(s1=555666.0, s2=888444.0, s3='Coimbra', myint=8)]

In [27]:
# CLEAN STRING COLUMN
df_clean = df
null_val = 'NONE_VALUE_'
print(f"Before: {df.count()}")
for col, dtype in df.dtypes:
    if 'string' not in dtype:
        print(f"{col} not string: {dtype}")
        print("--"*10)
        continue
    df_clean = df_clean.fillna(null_val, col)
    if df_clean.select(col).distinct().count() > 5:
        print(f"#### {col} (>5) ####")
        print(f"Before: {df_clean.count()}")
        df_clean = df_clean.filter(f"{col} != 'NONE_VALUE_'")
        print(f"After {col}: {df_clean.count()}\n")
    else:
        print(f"#### {col} (<5) ####")
    l = df_clean.select(col).distinct().rdd.map(lambda x: x[0])
    print(f"distinct count: {l.count()}\n")
    for v in sorted(l.collect()):
        print(v)
    print('\n'*4)
    print("--"*10)

Before: 717
#### comparable_rental_2_address (>5) ####
Before: 717
After comparable_rental_2_address: 710

distinct count: 312

1004 HEGEMAN AVENUE
1005 45 STREET
101 BRIGHTWATER COURT
1016 44 STREET
1017 44 STREET
104 2 PLACE
1044 EAST NEW YORK AVENUE
10515 SEAVIEW AVENUE
1054 BERGEN AVENUE
1058 BERGEN STREET
108 ATLANTIC AVENUE
108 DUPONT STREET
110 AVENUE C
110 EMPIRE BOULEVARD
110 REMSEN STREET
1102 PRESIDENT STREET
1106 PRESIDENT STREET
115 HENRY STREET
1159 CARROLL STREET
1159 EASTERN PARKWAY
117 KENT AVENUE
1177 EAST 98 STREET
118 GARFIELD PLACE
1182 PUTNAM AVENUE
120 OCEAN PARKWAY
1226 FLUSHING AVENUE
1229 AVENUE Y
1247 FLATBUSH AVENUE
1255 GATES AVENUE
126 4 AVENUE
127 GREEN STREET
1284 PACIFIC STREET
129 JORALEMON STREET
131 COLUMBIA STREET
131 JORALEMON STREET
1312 68 STREET
1358 64 STREET
136 MONTAGUE STREET
136 PROSPECT PARK WEST
1367 63 STREET
1367 STERLING PLACE
138 HOPKINSON AVENUE
140 LAFAYETTE AVENUE
141 JORALEMON STREET
142 ST PAUL'S PLACE
1420 OCEAN PARKWAY
144 NORT

In [29]:
df_clean2 = replaced_df
string_cols = [col for col, dtype in df_clean2.dtypes if 'string' in dtype]
df_clean2 = df_clean2.dropna(subset=string_cols)

In [26]:
df_clean2.coalesce(1).write.csv('df_cleaned', mode='overwrite', sep='\t', header=True)

In [32]:
df.count()

717

In [30]:
df_clean.count()

480

In [4]:
sc.stop()